In [72]:
#Lesion', presence or absence of skin lesion
#'Genetic Propensity', average propensity score due to genetic causes
#'Skin X test', result of the clinical test,
#'Skin color', binary value considering a dichotomy of skin color
#'Small size', size of the lesion is small
#'Mid size', size of the lesion is mid
#'Large size', size of the lesion
#'Mid', area of the lesion is mid
#'Small', area of the lesion is small
#'Large', area of the lesion is large
#'Doughnuts consumption', the average number of dougnuts consumed every day by the patient,
#'Fake/Real': the outcome to predict


In [44]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
dataset = pd.read_csv('train_A_derma.csv')
dataset.head()
X=dataset.values
y=X[:,-1]
X=X[:,:-1]


dataset=dataset.drop('Doughnuts consumption',axis=1)
#Let's try to fill the data.
dataset = dataset.dropna(thresh=dataset.shape[1] - 5 + 1) #drop rows with more than 5 isnan values

dataset['Fake/Real']=np.where(dataset['Fake/Real']=='real',1,0)

dataset['coincide'] = np.where((dataset['Small size'] == dataset['Small']) | 
                              (dataset['Large size'] == dataset['Large']) | 
                               (dataset['Mid size'] == dataset['Mid']), 1, 0)

#We have to fill the nan in a consistent way

mask =((dataset['Mid'] == 1) | (dataset['Small'] == 1) | (dataset['Large'] == 1) |
    (dataset['Mid size'] == 1) | (dataset['Small size'] == 1) | (dataset['Large size'] == 1))
dataset['Lession'].loc[mask] = dataset['Lession'].loc[mask].fillna(1)

#creiamo una funzione che mi completa i nan con gradient boosting: da genetic propensity in poi


dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 503 entries, 0 to 596
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  503 non-null    int64  
 1   Lession             488 non-null    float64
 2   Genetic Propensity  361 non-null    float64
 3   Skin X test         379 non-null    float64
 4   Skin color          380 non-null    float64
 5   Small size          369 non-null    float64
 6   Mid size            374 non-null    float64
 7   Large size          369 non-null    float64
 8   Mid                 384 non-null    float64
 9   Small               370 non-null    float64
 10  Large               366 non-null    float64
 11  Fake/Real           503 non-null    int32  
 12  coincide            503 non-null    int32  
dtypes: float64(10), int32(2), int64(1)
memory usage: 51.1 KB


C:\Users\Eddie\AppData\Local\Temp\ipykernel_9260\1451583161.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['Lession'].loc[mask] = dataset['Lession'].loc[mask].fillna(1)


In [47]:
model=xgb.XGBClassifier()
subset_data = dataset.dropna(subset='Skin color')
X = subset_data.drop('Skin color', axis=1)
y = subset_data['Skin color']  
model.fit(X, y)
X_missing = dataset.drop('Skin color', axis=1)  
missing_indices = dataset['Skin color'].isna() 
predictions = model.predict(X_missing)
dataset.loc[missing_indices, 'Skin color'] = predictions[missing_indices]

In [48]:
dataset['Skin X test']=dataset['Skin X test'].astype('Int64')
dataset['Skin X test']-=1

In [49]:
import xgboost as xgb
model = xgb.XGBClassifier()

dict=['Small size', 'Mid size', 'Large size', 'Mid', 'Small', 'Large','Skin X test']

for word in dict:
    subset_data = dataset.dropna(subset=[word])
    X = subset_data.drop(word, axis=1)  # Variabili indipendenti
    y = subset_data[word]  
    model.fit(X, y)
    X_missing = dataset.drop(word, axis=1)  
    missing_indices = dataset[word].isna() 
    predictions = model.predict(X_missing)
    dataset.loc[missing_indices, word] = predictions[missing_indices]
    

In [50]:
mean = dataset['Genetic Propensity'].mean()
std = dataset['Genetic Propensity'].std()
missing_indices = dataset['Genetic Propensity'].isna()
random_values = np.random.normal(mean, std, missing_indices.sum())
dataset.loc[missing_indices, 'Genetic Propensity'] = random_values

In [51]:
dataset['Lession']=dataset['Lession'].fillna(0)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 503 entries, 0 to 596
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Lession             503 non-null    float64
 1   Genetic Propensity  503 non-null    float64
 2   Skin X test         503 non-null    Int64  
 3   Skin color          503 non-null    float64
 4   Small size          503 non-null    float64
 5   Mid size            503 non-null    float64
 6   Large size          503 non-null    float64
 7   Mid                 503 non-null    float64
 8   Small               503 non-null    float64
 9   Large               503 non-null    float64
 10  Fake/Real           503 non-null    int32  
 11  coincide            503 non-null    int32  
dtypes: Int64(1), float64(9), int32(2)
memory usage: 47.6 KB


In [52]:
y=dataset['Fake/Real']
dataset=dataset.drop('Fake/Real',axis=1)
X=dataset.values

In [11]:
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn import neighbors

                               
X_train,X_val,y_train,y_val = model_selection.train_test_split(X,y,train_size=0.8,random_state=42)

#model = RandomForestClassifier(n_estimators=10, random_state=42)
model = xgb.XGBClassifier()
model.fit(X_train,y_train)
y_pred=model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(accuracy)

0.801980198019802


In [14]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

# Crea un modello XGBoost
model = XGBClassifier()

# Definisci una griglia di parametri da esplorare
param_grid = {
    'n_estimators': [5,10,15,20,25,30,35,40],
    'learning_rate': [0.5,1],
    'max_depth': [3, 4, 5],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2],
}
X_train,X_val,y_train,y_val = model_selection.train_test_split(X,y,train_size=0.9,random_state=42)

# Crea un oggetto GridSearchCV per la ricerca degli iperparametri
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)

# Addestra il modello con la griglia di parametri
grid_search.fit(X_train, y_train)

# Ottieni il miglior modello
best_model = grid_search.best_estimator_

# Esegui le previsioni sul set di convalida
y_pred = best_model.predict(X_val)

# Calcola l'accuratezza sul set di convalida
accuracy = accuracy_score(y_val, y_pred)

# Stampa i parametri ottimali e l'accuratezza
print("Parametri ottimali:", grid_search.best_params_)
print("Accuratezza sul set di convalida:", accuracy)


Parametri ottimali: {'colsample_bytree': 0.9, 'gamma': 0, 'learning_rate': 0.5, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 5, 'subsample': 0.9}
Accuratezza sul set di convalida: 0.7647058823529411


In [10]:
from sklearn.preprocessing import StandardScaler
random_state=np.arange(43)
accuracy=[]
scaler = StandardScaler()

for i in random_state:
    X_train,X_val,y_train,y_val = model_selection.train_test_split(X,y,train_size=0.9,random_state=i)
    X_train= scaler.fit_transform(X_train)
    model = xgb.XGBClassifier()
    model.fit(X_train, y_train)
    X_val=scaler.transform(X_val)
    y_pred=model.predict(X_val)
    accuracy.append(accuracy_score(y_val, y_pred))
    
accuracy=np.array(accuracy)
print(accuracy.max())
print(accuracy.mean())

0.9411764705882353
0.8071135430916551


In [16]:
import pandas as pd
test_set= pd.read_csv('test_A_derma.csv')
test_set=test_set.drop('Doughnuts consumption',axis=1)
#Let's try to fill the data.

test_set['coincide'] = np.where((test_set['Small size'] == test_set['Small']) | 
                              (test_set['Large size'] == test_set['Large']) | 
                               (test_set['Mid size'] == test_set['Mid']), 1, 0)

#We have to fill the nan in a consistent way

mask =((test_set['Mid'] == 1) | (test_set['Small'] == 1) | (test_set['Large'] == 1) |
    (test_set['Mid size'] == 1) | (test_set['Small size'] == 1) | (test_set['Large size'] == 1))
test_set['Lession'].loc[mask] = test_set['Lession'].loc[mask].fillna(1)

C:\Users\Eddie\AppData\Local\Temp\ipykernel_9260\4162459978.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set['Lession'].loc[mask] = test_set['Lession'].loc[mask].fillna(1)


In [17]:
import xgboost as xgb
model = xgb.XGBClassifier()
subset_data = test_set.dropna(subset='Skin color')
X = subset_data.drop('Skin color', axis=1)
y = subset_data['Skin color']  
model.fit(X, y)
X_missing = test_set.drop('Skin color', axis=1)  
missing_indices = test_set['Skin color'].isna() 
predictions = model.predict(X_missing)
test_set.loc[missing_indices, 'Skin color'] = predictions[missing_indices]

In [18]:
test_set['Skin X test']=test_set['Skin X test'].astype('Int64')
test_set['Skin X test']-=1

In [19]:
import xgboost as xgb
model = xgb.XGBClassifier()

dict=['Small size', 'Mid size', 'Large size', 'Mid', 'Small', 'Large','Skin X test']

for word in dict:
    subset_data = test_set.dropna(subset=[word])
    X = subset_data.drop(word, axis=1)  # Variabili indipendenti
    y = subset_data[word]  
    model.fit(X, y)
    X_missing = test_set.drop(word, axis=1)  
    missing_indices = test_set[word].isna() 
    predictions = model.predict(X_missing)
    test_set.loc[missing_indices, word] = predictions[missing_indices]

In [20]:
mean = test_set['Genetic Propensity'].mean()
std = test_set['Genetic Propensity'].std()
missing_indices = test_set['Genetic Propensity'].isna()
random_values = np.random.normal(mean, std, missing_indices.sum())
test_set.loc[missing_indices, 'Genetic Propensity'] = random_values

In [21]:
test_set['Lession']=test_set['Lession'].fillna(0)


In [60]:
X=dataset.values

#test_set=test_set.drop(columns='Id', axis=1)
X_test=test_set.values
model = xgb.XGBClassifier()
model.fit(X,y)
y_pred=model.predict(X_test)

In [62]:
print(y_pred)

[0 1 0 1 1 0 1 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 0 1 0 0
 0 0 1 0 0 0 1 0 1 1 1 0 0 1 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1
 0 0 0 0 1 1 1 0 1 1 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 1 0 1 1 0 1 0 1 0 1 0 1
 0 0 1 1 1 1 0 0 1 1 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 1 0 1 0 1 0 0 1 1 1 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 1 1 1 0 0 1 1 1 0 0 1 1 1
 0 0 1 0 0 0 1 0 0 1 1 1 0 0 0 1 0 1 0 0 0 1 0 1 0 0 1 1 1 1 0 0 0 0 1 0 1
 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 1 1 1 1 1 0 0 0 1 1 1 0]


In [63]:
y_pred=pd.Series(np.where(y_pred==1, 'real', 'fake'))

In [65]:
print(y_pred)
y_pred.to_csv('Y_pred_testA_Eddie.csv')

0      fake
1      real
2      fake
3      real
4      real
       ... 
251    fake
252    real
253    real
254    real
255    fake
Length: 256, dtype: object


In [66]:
Eddie_A = pd.read_csv('Y_pred_testA_Eddie.csv').iloc[:256,-1]

eddie = pd.read_csv('y_prediction_v7.csv').iloc[256:,-1]

y_prediction = pd.concat([Eddie_A, eddie]).reset_index(drop=True).reset_index()
y_prediction.columns = ['Id','Prediction']

In [68]:
y_prediction.to_csv('y_prediction_v22.csv', sep=',',index=False)